Projeto Final de Spark

Campanha Nacional de Vacinação contra Covid-19

Nível Avançado:
Replicar as visualizações do site “https://covid.saude.gov.br/”, porém acessando diretamente a API de Elastic.

In [1]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
# Criação da string com a URL de destino da API Elastic para fonte de dados MSCOVIDBR
es_url = "https://imunizacao-es.saude.gov.br/desc-imunizacao/_search"
# Agora vamos ler os dados do arquivo csv com o seguinte parâmetro da API:
# {
#  "size": 10000
# }
mscovidbrinputDf = spark.read.option('header', 'true').csv('/user/jes/data/mscovidbr_input.csv')
# E criamos uma tabela temporária agora usando o 'mscovidbrinputDf'
mscovidbrinputDf.createOrReplaceTempView('mscovidbrinputtbl')
# Agora criamos o mapa de parâmetros para passar para a fonte de dados REST
prmsMSCOVIDBR = { 'url' : es_url, 'input' : 'mscovidbrinputtbl', 'userId' : 'imunizacao_public', 'userPassword' : 'qlto5t&7r_@+#Tlstigi', 'method' : 'GET', 'readTimeout' : '10000', 'connectionTimeout' : '2000', 'partitions' : '10'}
# Agora criamos o Dataframe que contém o resultado da chamada para a API com o parâmetro
mscovidbrDf = spark.read.format("org.apache.dsext.spark.datasource.rest.RestDataSource").options(**prmsMSCOVIDBR).load()
# Nós inspecionamos a estrutura dos resultados retornados.
mscovidbrDf.printSchema()

root
 |-- output: struct (nullable = true)
 |    |-- _shards: struct (nullable = true)
 |    |    |-- failed: long (nullable = true)
 |    |    |-- skipped: long (nullable = true)
 |    |    |-- successful: long (nullable = true)
 |    |    |-- total: long (nullable = true)
 |    |-- hits: struct (nullable = true)
 |    |    |-- hits: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- _id: string (nullable = true)
 |    |    |    |    |-- _index: string (nullable = true)
 |    |    |    |    |-- _score: double (nullable = true)
 |    |    |    |    |-- _source: struct (nullable = true)
 |    |    |    |    |    |-- @timestamp: string (nullable = true)
 |    |    |    |    |    |-- @version: string (nullable = true)
 |    |    |    |    |    |-- data_importacao_rnds: string (nullable = true)
 |    |    |    |    |    |-- document_id: string (nullable = true)
 |    |    |    |    |    |-- estabelecimento_municipio_codigo: string (nu

In [2]:
# Criamos o Dataframe 'mscovidbrDf_vacina' utilizando a função 'explode' 
# para retornar uma nova linha para cada elemento no array com dados dos pacientes vacinados.
mscovidbrDf_vacina = mscovidbrDf.select(F.explode(F.col("output.hits.hits").alias("paciente")))
# Nós inspecionamos a estrutura dos resultados retornados.
mscovidbrDf_vacina.printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- _id: string (nullable = true)
 |    |-- _index: string (nullable = true)
 |    |-- _score: double (nullable = true)
 |    |-- _source: struct (nullable = true)
 |    |    |-- @timestamp: string (nullable = true)
 |    |    |-- @version: string (nullable = true)
 |    |    |-- data_importacao_rnds: string (nullable = true)
 |    |    |-- document_id: string (nullable = true)
 |    |    |-- estabelecimento_municipio_codigo: string (nullable = true)
 |    |    |-- estabelecimento_municipio_nome: string (nullable = true)
 |    |    |-- estabelecimento_razaoSocial: string (nullable = true)
 |    |    |-- estabelecimento_uf: string (nullable = true)
 |    |    |-- estabelecimento_valor: string (nullable = true)
 |    |    |-- estalecimento_noFantasia: string (nullable = true)
 |    |    |-- id_sistema_origem: string (nullable = true)
 |    |    |-- paciente_dataNascimento: string (nullable = true)
 |    |    |-- paciente_endereco_cep: string 

In [3]:
# Agora estamos prontos para aplicar SQL ou qualquer outro processamento nos resultados
# Visualiações:
# 1ª - Dos dados de vacinados por UF, município, sexo e idade(média, menor e maior)
mscovidbrDf_vacina_sexo_idade = mscovidbrDf_vacina.groupBy(F.col("col._source.estabelecimento_uf").alias("UF"), \
                                                   F.col("col._source.estabelecimento_municipio_nome").alias("Municipio"), \
                                                   F.col("col._source.paciente_enumSexoBiologico").alias("Sexo") \
                                                   ).agg(F.format_number(F.avg(F.col("col._source.paciente_idade")),2).alias("Media_de_idade"), \
                                                       F.min(F.col("col._source.paciente_idade")).alias("Idade_menor"), \
                                                       F.max(F.col("col._source.paciente_idade")).alias("Idade_maior") \
                                                        ).sort(F.asc("UF"))
mscovidbrDf_vacina_sexo_idade.show(truncate=False)

+---+-------------------------+----+--------------+-----------+-----------+
|UF |Municipio                |Sexo|Media_de_idade|Idade_menor|Idade_maior|
+---+-------------------------+----+--------------+-----------+-----------+
|AC |PORTO WALTER             |F   |84.00         |84         |84         |
|AC |RIO BRANCO               |F   |49.57         |19         |101        |
|AC |ASSIS BRASIL             |M   |31.00         |20         |42         |
|AC |SENA MADUREIRA           |M   |94.00         |94         |94         |
|AC |RIO BRANCO               |M   |55.33         |32         |92         |
|AC |FEIJO                    |F   |81.00         |80         |82         |
|AC |FEIJO                    |M   |86.00         |77         |91         |
|AC |SENA MADUREIRA           |F   |90.00         |84         |96         |
|AL |PARICONHA                |M   |61.00         |51         |72         |
|AL |PIACABUCU                |F   |55.00         |55         |55         |
|AL |PARICON

In [ ]:
# Salvando a primeira visualização como tabela Hive
mscovidbrDf_vacina_sexo_idade.write.saveAsTable("vacina_sexo_idade")

In [4]:
# 2ª - Dos dados de vacinados por UF, município, vacina e dose aplicada
mscovidbrDf_vacina_dose = mscovidbrDf_vacina.groupBy(F.col("col._source.estabelecimento_uf").alias("UF"), \
                                                   F.col("col._source.estabelecimento_municipio_nome").alias("Municipio"), \
                                                   F.col("col._source.vacina_nome").alias("Vacina"), \
                                                   F.col("col._source.vacina_descricao_dose").alias("Dose") \
                                                   ).agg(F.count(F.col("col._source.vacina_descricao_dose")).alias("Qtd_aplicacoes") \
                                                        ).sort(F.asc("UF"))
mscovidbrDf_vacina_dose.show(truncate=False)

+---+-------------------+-----------------------------------+-------+--------------+
|UF |Municipio          |Vacina                             |Dose   |Qtd_aplicacoes|
+---+-------------------+-----------------------------------+-------+--------------+
|AC |TARAUACA           |Vacina Covid-19 - Covishield       |1ª Dose|1             |
|AC |SANTA ROSA DO PURUS|Covid-19-Coronavac-Sinovac/Butantan|2ª Dose|1             |
|AC |RIO BRANCO         |Covid-19-Coronavac-Sinovac/Butantan|1ª Dose|2             |
|AC |RIO BRANCO         |Vacina Covid-19 - Covishield       |1ª Dose|11            |
|AC |FEIJO              |Covid-19-Coronavac-Sinovac/Butantan|2ª Dose|1             |
|AC |CRUZEIRO DO SUL    |Covid-19-Coronavac-Sinovac/Butantan|2ª Dose|2             |
|AL |PAO DE ACUCAR      |Covid-19-Coronavac-Sinovac/Butantan|1ª Dose|1             |
|AL |BOCA DA MATA       |Covid-19-Coronavac-Sinovac/Butantan|1ª Dose|3             |
|AL |JACARE DOS HOMENS  |Covid-19-Coronavac-Sinovac/Butantan|1ª D

In [ ]:
# Salvando a segunda visualização com formato parquet e compressão snappy
mscovidbrDf_vacina_dose.write.save("/user/jes/vacina_parquet")

In [5]:
!hdfs dfs -ls /user/jes/vacina_parquet

Found 26 items
-rw-r--r--   2 root supergroup          0 2021-07-12 21:33 /user/jes/vacina_parquet/_SUCCESS
-rw-r--r--   2 root supergroup       2300 2021-07-12 21:33 /user/jes/vacina_parquet/part-00000-ffe6d4e3-bfb2-4002-9686-2ce147139853-c000.snappy.parquet
-rw-r--r--   2 root supergroup       2066 2021-07-12 21:33 /user/jes/vacina_parquet/part-00001-ffe6d4e3-bfb2-4002-9686-2ce147139853-c000.snappy.parquet
-rw-r--r--   2 root supergroup       1660 2021-07-12 21:33 /user/jes/vacina_parquet/part-00002-ffe6d4e3-bfb2-4002-9686-2ce147139853-c000.snappy.parquet
-rw-r--r--   2 root supergroup       4482 2021-07-12 21:33 /user/jes/vacina_parquet/part-00003-ffe6d4e3-bfb2-4002-9686-2ce147139853-c000.snappy.parquet
-rw-r--r--   2 root supergroup       2011 2021-07-12 21:33 /user/jes/vacina_parquet/part-00004-ffe6d4e3-bfb2-4002-9686-2ce147139853-c000.snappy.parquet
-rw-r--r--   2 root supergroup       1595 2021-07-12 21:33 /user/jes/vacina_parquet/part-00005-ffe6d4e3-bfb2-4002-9686-2ce147139853-

In [6]:
# 3ª - Dos dados de vacinados por UF, município, raça, sexo, vacina e dose aplicada
mscovidbrDf_vacina_dose_sexoraca = mscovidbrDf_vacina.groupBy(F.col("col._source.estabelecimento_uf").alias("UF"), \
                                                   F.col("col._source.estabelecimento_municipio_nome").alias("Municipio"), \
                                                   F.col("col._source.vacina_nome").alias("Vacina"), \
                                                   F.col("col._source.vacina_descricao_dose").alias("Dose"), \
                                                   F.col("col._source.paciente_enumSexoBiologico").alias("Sexo"), \
                                                   F.col("col._source.paciente_racaCor_valor").alias("Raca") \
                                                   ).agg(F.count(F.col("col._source.paciente_racaCor_valor")).alias("Qtd_aplicacoes"), \
                                                         F.min(F.col("col._source.paciente_idade")).alias("Idade_menor"), \
                                                         F.max(F.col("col._source.paciente_idade")).alias("Idade_maior") \
                                                        ).sort(F.asc("UF"))
mscovidbrDf_vacina_dose_sexoraca.show()

+---+-------------------+--------------------+-------+----+--------------+--------------+-----------+-----------+
| UF|          Municipio|              Vacina|   Dose|Sexo|          Raca|Qtd_aplicacoes|Idade_menor|Idade_maior|
+---+-------------------+--------------------+-------+----+--------------+--------------+-----------+-----------+
| AC|         RIO BRANCO|Vacina Covid-19 -...|1ª Dose|   F|SEM INFORMACAO|             2|         40|         73|
| AC|SANTA ROSA DO PURUS|Covid-19-Coronava...|2ª Dose|   F|      INDIGENA|             1|         20|         20|
| AC|           TARAUACA|Vacina Covid-19 -...|1ª Dose|   F|       AMARELA|             1|         80|         80|
| AC|         RIO BRANCO|Vacina Covid-19 -...|1ª Dose|   F|       AMARELA|             1|         53|         53|
| AC|         RIO BRANCO|Vacina Covid-19 -...|1ª Dose|   M|       AMARELA|             3|         79|         83|
| AC|    CRUZEIRO DO SUL|Covid-19-Coronava...|2ª Dose|   F|       AMARELA|             1

In [7]:
# 4ª - Da quantidade de aplicações por vacina
mscovidbrDf_vacina_dose_aplicacoes = mscovidbrDf_vacina.groupBy(F.col("col._source.vacina_nome").alias("key") \
                                                   ).agg(F.count(F.col("col._source.vacina_nome")).cast("string").alias("value") \
                                                        )
mscovidbrDf_vacina_dose_aplicacoes.show(truncate=False)

+-----------------------------------+-----+
|key                                |value|
+-----------------------------------+-----+
|Covid-19-Coronavac-Sinovac/Butantan|8651 |
|Vacina Covid-19 - Covishield       |1349 |
+-----------------------------------+-----+



In [8]:
mscovidbrDf_vacina_dose_aplicacoes.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = false)



In [ ]:
# Salvando a quarta visualização em um tópico no Kafka
mscovidbrDf_vacina_dose_aplicacoes.write \
                                  .format("kafka") \
                                  .option("kafka.bootstrap.servers", "kafka:9092") \
                                  .option("topic", "topic-kvvacina") \
                                  .save()

Referência:

Neste projeto foi utilizada a biblioteca "Rest Data Source for Apache Spark" (https://github.com/sourav-mazumder/Data-Science-Extensions/tree/master/spark-datasource-rest)

Esta é uma biblioteca para chamar serviços/APIs baseados em REST para vários conjuntos de parâmetros de entrada em paralelo e agrupar os resultados, retornados pelo serviço REST, em um Dataframe.